# Market Overlap

### SCENARIO

The scenario is: A ficticious company named 'Alpha' is interested in acquiring another ficticious company named 'Beta'. They both belong to the hotelling industry, more specifically they are marketplaces for hotels to announce their accomodation details as well as handle all the booking process with travellers.

### UNDERSTANDING THE COMPANIES

__Alpha__ offer hotels in destinations worldwide. They've entered Brazil's market recently and have been investing in Facebook and Google Ads to get more hotels to publish in their marketplace. However, the CAC (customer acquisition cost) to get more hotels on board is too high and after several weeks trying to improve performance Alpha's board have decided to look for other options of increasing the number of hotels in their marketplace.

__Beta__ on the other hand, operates only in Brazil and although has a smaller scale than Alpha overall, it has a considerable amount of hotels already on board and operating.

### WHY THE MARKET OVERLAP ANALYSIS

Seeking a way to reduce the CAC, Alpha has made a move to buy Beta with all it's hotels and marketplace service. Even though Beta has shown interest on the deal, Alpha still want's to look at some real data in order to reach the conclusion if the CAC would really be lower than their current marketing investments.

As of now, both companies does not know if their hotels are unique to them or actually common between both, as its a common practice among hotels to publish their accomodations in different marketplaces.